# Planetary Nebula Production <a class="tocSkip">
    
This notebook is used to test and showcase the results of my first project. I use spectroscopic data from the [Multi Unit Spectroscopic Explorer](https://www.eso.org/sci/facilities/develop/instruments/muse.html) (MUSE) that has been observed as part of the [PHANGS](https://sites.google.com/view/phangs/home) collaboration.
    
I will use a set of line maps of emission lines to identify Planetary Nebula in the data an measure their brightness. This can then be used to fit an empiric relation and hence measure the distance to the galaxy.
    
This notebook is used for developement. Final code is moved to the `pymuse` packge in the `src` folder. Any production scripts reside in the `scripts` folder.

## Preparation
 
### Load Basic Packages
    
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

# this imports packages like numpy or astropy 
from pymuse.packages import *
# constants that are used across multiple functions
from pymuse.constants import tab10, single_column, two_column

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,format='%(levelname)s: %(message)s',level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

### Read in data

this uses the `ReadLineMaps` class from the `pymuse.io` module. To use it, we first need to specify the path to the data folder

IC5332, NGC628, NGC1087, NGC1365, NGC1512, NGC1566, NGC1672, NGC2835, NGC3351, NGC3627, NGC4254, NGC4535, NGC5068

In [ ]:
from pymuse.io import ReadLineMaps

#with open(basedir / 'data' / 'interim' / 'parameters.json') as json_file:
#    parameters = json.load(json_file)
with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)
    
# table to save all results
results = ascii.read(basedir/'data'/'interim'/ 'results.txt',format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
results.add_index('name')    

name = 'NGC7496'

# first we need to specify the path to the raw data
basedir = Path('..')
data_raw = basedir / 'data' / 'raw' / 'MUSE' / 'DR2'
#data_raw = Path('d:\downloads\MUSEDAP')
#data_ext = Path('g:\Archive')

extensions = ['OIII5006', 'HA6562', 'NII6583', 'SII6716']

# read in the data we will be working with and print some information
galaxy = ReadLineMaps(data_raw,name,extensions,**parameters[name])

### Mask unwanted regions

In [ ]:
from pymuse.auxiliary import circular_mask
from pymuse.plot.plot import create_RGB
    
mask = np.zeros(galaxy.shape,dtype=bool)
mask |= galaxy.star_mask.astype(bool)

# define masks as slices
masks = {
 'NGC1300' : circular_mask(*galaxy.shape,radius=50),
 'NGC1365' : circular_mask(*galaxy.shape,(720,420),radius=200),
 'NGC1433' : circular_mask(*galaxy.shape,radius=70),
 'NGC1512' : circular_mask(*galaxy.shape,radius=70),
 'NGC1566' : circular_mask(*galaxy.shape,(450,450),radius=100)|circular_mask(*galaxy.shape,(350,150),radius=180),
 'NGC1672' : circular_mask(*galaxy.shape,(600,310),radius=100),
 'NGC3627' : circular_mask(*galaxy.shape,(330,740),radius=100),
 'NGC3351' : circular_mask(*galaxy.shape,radius=200),
 'NGC4321' : circular_mask(*galaxy.shape,(550,450),radius=60),
 'NGC4535' : circular_mask(*galaxy.shape,(300,520),radius=100)
}

mask[masks.get(galaxy.name,(slice(-1,0),slice(-1,0)))] = True

#img = galaxy.OIII5006_DAP.copy()
img = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.6,1,0.6],percentile=[95,99.,95])
img[mask,...] = (1,1,1)

fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot()

#norm = simple_norm(galaxy.OIII5006_DAP,clip=False,max_percent=95)
ax.imshow(img,origin='lower')
plt.show()

## Source Detection

In [ ]:
from photutils import DAOStarFinder            # DAOFIND routine to detect sources
from photutils import IRAFStarFinder           # IRAF starfind routine to detect star

from pymuse.detection import detect_unresolved_sources

In [ ]:
# we include all sources in this step and reject bad ones later
sharplo   = 0.0 #galaxy.sharplo
sharphi   = 1 #galaxy.sharphi
roundness = 1 #galaxy.roundness

sources = detect_unresolved_sources(galaxy,
                                    'OIII5006_DAP',
                                    StarFinder=DAOStarFinder,
                                    threshold=galaxy.threshold,
                                    exclude_region=mask,
                                    oversize=1.,
                                    roundlo=-roundness,
                                    roundhi=roundness,
                                    sharplo=sharplo,
                                    sharphi=sharphi,
                                    save=False)

## Completeness limit

In [ ]:
from pymuse.detection import completeness_limit

In [ ]:
with open(basedir / 'data' / 'interim' / 'parameters.yml') as yml_file:
    parameters = yaml.load(yml_file,Loader=yaml.FullLoader)
    
setattr(galaxy,'binsize',parameters[galaxy.name]['binsize'])
setattr(galaxy,'mu',parameters[galaxy.name]['mu'])
setattr(galaxy,'alpha',parameters[galaxy.name]['power_index'])
setattr(galaxy,'completeness_limit',parameters[galaxy.name]['completeness_limit'])
setattr(galaxy,'roundness',parameters[galaxy.name]['roundness'])
setattr(galaxy,'sharplo',parameters[galaxy.name]['sharplo'])
setattr(galaxy,'sharphi',parameters[galaxy.name]['sharphi'])
setattr(galaxy,'threshold',parameters[galaxy.name]['threshold'])
setattr(galaxy,'zoomin',parameters[galaxy.name]['zoomin'])

print(f'for {galaxy.name}')
mock_sources = completeness_limit(
                    galaxy,
                    'OIII5006_DAP',
                    DAOStarFinder,
                    threshold=galaxy.threshold,
                    iterations=1,
                    stars_per_mag=50,
                    exclude_region=mask,
                    roundlo=-galaxy.roundness,
                    roundhi=galaxy.roundness,
                    sharplo=galaxy.sharplo,
                    sharphi=galaxy.sharphi,
                    exclude_border=True,
                    plot=False
                                 )

In [ ]:
for x_std in np.unique(mock_sources['x_stddev']):
    sub_sample = mock_sources[mock_sources['x_stddev']==x_std]
    
    for a in np.unique(sub_sample['amplitude']):
        sub_sub = sub_sample[sub_sample['amplitude']==a]

        found = np.sum(sub_sub['sep']<0.5)
        total = len(sub_sub)
        print(f'a={a:.2f}: {found} of {total} recovered ({found/total*100:.2f})')
        
    print(30*'-')

## Flux measurement

In the previous step we detected potential PN candidates by their [OIII] emission. This means we know their position but lack exact flux measurments. In this section we measure the flux of the identified objects in different emission lines that are used in later steps. 

In [ ]:
from pymuse.photometry import measure_flux 

In [ ]:
Rv  = 3.1
Ebv = 0.062
aperture_size = galaxy.aperturesize

flux = measure_flux(galaxy,
                    sources,
                    alpha=galaxy.power_index,
                    Rv=Rv,
                    Ebv=Ebv,
                    extinction='MW',
                    background='local',
                    aperture_size=aperture_size)

# calculate astronomical coordinates for comparison

# calculate magnitudes from measured fluxes
flux['mOIII'] = -2.5*np.log10(flux['OIII5006']*1e-20) - 13.74
flux['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

## Emission line diagnostics    

In [ ]:
from pymuse.analyse import emission_line_diagnostics

print(f'emission line diagnostics for {galaxy.name}')
print(f'mu={galaxy.mu:.2f}, cl={galaxy.completeness_limit}')
tbl = emission_line_diagnostics(flux,galaxy.mu,galaxy.completeness_limit) 

# create additional columns that are needed for the classification
tbl['sharp'] = sources['sharpness']
tbl['round'] = sources['roundness2']
tbl['SkyCoord'] = SkyCoord.from_pixel(tbl['x'],tbl['y'],galaxy.wcs)

tbl['exclude'] = False

cut=0
slow    = galaxy.sharplo  
shigh   = galaxy.sharphi 
r       = galaxy.roundness 
if cut>0:
    logger.warning('you are using a cut')
    
#slope = []
#for row in tbl:
#    star = Cutout2D(galaxy.OIII5006, (row['x'],row['y']), u.Quantity((size, size), u.pixel),wcs=galaxy.wcs)
#    profile = radial_profile(star.data,star.input_position_cutout)
#    slope.append(np.sum(np.ediff1d(profile)>0) / len(profile))
#tbl['slope'] = slope    

# table contains all detected objects. here we mask all undesired objects.
c_shape = ((tbl['sharp']>slow) & (tbl['sharp']<shigh) & (np.abs(tbl['round'])<r)) #& (tbl['OIII5006']>10*np.abs(tbl['OIII5006_bkg_local']))
c_PN    = (tbl['type']=='PN')
c_SNR   = (tbl['SNRorPN'] & (tbl['type']=='SNR'))
c_AV    = ((tbl['Av']<0.4) | np.isnan(tbl['Av']))
c_cut   = (cut<tbl['mOIII'])
c_detec = tbl['OIII5006_detection'] 
c_limit = (tbl['mOIII']<galaxy.completeness_limit) 

In [ ]:
from pymuse.plot.plot import single_cutout

tmp = tbl[c_shape & c_cut & (c_PN|c_SNR) & c_detec & ~tbl['exclude']].copy()
tmp.sort('mOIII')

s = 32
for row in tmp[:5]:
    try:
        ax1,ax2,ax3 = single_cutout(galaxy,row['x'],row['y'],size=s,percentile=95,aperture_size=aperture_size*row['fwhm']/2)
    except:
        print(f'error for {row["id"]}')
    ax1.set_title(row['id'])
    ax2.set_title(f'mOIII = {row["mOIII"]:.2f}')
    ax3.set_title(f'r={row["round"]:.2f}, s={row["sharp"]:.2f}')

In [ ]:
# remove all objects defined here from the sample
# define masks as slices
exclude = {
 'IC5332'  : [2376,1755],
 'NGC0628' : [318,934],
 'NGC1087' : [1207,772,464,158],
 #'NGC1087' : [1130,723,1402],
 'NGC1300' : [1234,1236],
 'NGC1365' : [1185,1812,1482,1662,100,1638,1610,71],
 'NGC1385' : [416,418,518,148,485,302,526,180],
 'NGC1433' : [9846],
 'NGC1512' : [5525],
 'NGC1566' : [326],
 'NGC1672' : [347,616,233,293,427,127,231],
 'NGC2835' : [268,239],
 'NGC3627' : [467,484],
 'NGC4254' : [2369,514],
 'NGC4303' : [606,435,1392,508],
 'NGC4535' : [821],
 'NGC5068' : [153],
 'NGC7496' : [729,600,169,171,42,597]
}

indices = np.where(np.in1d(tbl['id'], exclude.get(galaxy.name,[])))[0]
tbl['exclude'][indices]=True

## Planetary nebula luminosity function

In [ ]:
from pymuse.analyse import MaximumLikelihood1D, PNLF, pnlf
from pymuse.plot.pnlf import plot_pnlf
from pymuse.auxiliary import uncertainties

binsize = galaxy.binsize

criteria = c_shape & c_cut & (c_PN) & c_detec & ~tbl['exclude']
data = tbl[np.where(criteria & c_limit)]['mOIII']
err = tbl[np.where(criteria & c_limit)]['dmOIII']
    
print(f'analysing {galaxy.name}')
print(f'completeness limit = {galaxy.completeness_limit}, binsize = {binsize}')
fitter = MaximumLikelihood1D(pnlf,data,mhigh=galaxy.completeness_limit,Mmax=-4.47)
galaxy.mu,dp,dm = fitter([28])
print(f'sample table: {parameters[name]["mu"]}')
print('{:.2f} + {:.2f} - {:.2f}'.format(*uncertainties(galaxy.mu,dp,dm)))

#Plot PNLF
filename = basedir / 'reports' / f'{galaxy.name}' / f'{galaxy.name}_PNLF'
axes = plot_pnlf(tbl[criteria]['mOIII'],
                 galaxy.mu,
                 galaxy.completeness_limit,
                 binsize=binsize,
                 #mhigh=29,
                 filename=filename,
                 color=tab10[0])


### Compare to literature

In [ ]:
from pymuse.plot.pnlf import compare_distances
print(galaxy.name)
filename = basedir / 'reports' / galaxy.name / f'{galaxy.name}_distances'
distances = compare_distances(galaxy.name,galaxy.mu,dp,dm,filename=filename)

### Save catalogue to file

In [ ]:
from pymuse.io import write_LaTeX

filename = basedir / 'data' / 'catalogues' 
write_LaTeX(tbl[c_shape &  c_detec & c_limit],galaxy,filename)

In [ ]:
# this saves the entire table  
tbl_out = tbl[c_shape & (tbl['type']!='NaN') & c_detec]
skycoord = SkyCoord.from_pixel(tbl_out['x'],tbl_out['y'],galaxy.wcs)
tbl_out['RaDec'] = skycoord.to_string(style='hmsdms',precision=2)

for col in tbl_out.colnames:
    if col not in ['id','RaDec','type','SNRorPN','SkyCoord']:
        if not col.endswith('detection'):
            tbl_out[col].info.format = '%.3f' 
            
filename = basedir / 'data' / 'catalogues' / f'{galaxy.name}_nebulae.txt'

with open(filename,'w',newline='\n') as f:
    ascii.write(tbl_out,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')
print(f'{len(tbl_out)} objects saved to ' + str(filename))   

### Visualize the result of the classification

In [ ]:
from pymuse.plot.pnlf import plot_emission_line_ratio

for t in ['PN','SNR','HII']:
    print(f"{t}: v_sig = {np.nanmean(tbl[(tbl['type']==t) &  (tbl['mOIII']<galaxy.completeness_limit) & (tbl['v_SIGMA_S/N']>9)]['v_SIGMA']):.2f}")

filename = basedir / 'reports' / galaxy.name / f'{galaxy.name}_emission_line'
plot_emission_line_ratio(tbl[c_shape & c_detec & c_limit],galaxy.mu,completeness=galaxy.completeness_limit,filename=filename)


In [ ]:
from pymuse.plot.plot import plot_sky_with_detected_stars
positions = np.transpose((sources['x'], sources['y']))
plot_sky_with_detected_stars(data=galaxy.OIII5006_DAP,
                             wcs=galaxy.wcs,
                             positions=positions,
                             filename=basedir/'reports'/galaxy.name/'sources.pdf')

In [ ]:
from pymuse.plot.classification import classification_map
#parameters[galaxy.name]['zoomin'] = [400,500,400]
print(galaxy.name)
filename = basedir / 'reports' / galaxy.name / f'{galaxy.name}_detections_classification.pdf'
classification_map(galaxy,parameters,tbl[c_shape & c_detec & c_limit],filename)

### PN per stellar mass

In [ ]:
xmin,xmax = 0,1000
bins = np.arange(xmin,xmax,(xmax-xmin)/10)
n,bins = np.histogram(galaxy.stellar_mass.flatten(),bins=bins)
n = n/np.sum(n)

fig, ax = plt.subplots(figsize=(two_column,two_column/1.618))

ax.bar((bins[1:]+bins[:-1])/2,n,width=(bins[1]-bins[0])/1.1)
ax.set(xlabel=r'stellar mass density / $M_\odot\; \mathrm{pc}^{-2}$',ylabel='count',xlim=[xmin,xmax])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(two_column,two_column/1.618))

hist, bins = np.histogram(tbl[np.where(criteria & c_limit)]['stellar_mass'],bins=bins)
hist = hist/n

ax.bar((bins[1:]+bins[:-1])/2,hist,width=(bins[1]-bins[0])/1.1)
ax.set(xlabel=r'stellar mass density / $M_\odot\; \mathrm{pc}^{-2}$',ylabel=r'$N_{PN}$',xlim=[xmin,xmax])
plt.show()

### With and without SNR

In [ ]:
from pymuse.analyse import MaximumLikelihood1D, PNLF, pnlf
from pymuse.plot.pnlf import plot_pnlf

criteria1 = c_shape & c_cut & (c_PN) & c_detec & ~tbl['exclude']
data1 = tbl[np.where(c_limit & criteria1)]['mOIII']
err1  = tbl[np.where(c_limit & criteria1)]['dmOIII']

criteria2 = c_shape & c_cut & (c_PN|c_SNR)  & c_detec & ~tbl['exclude']
data2 = tbl[np.where(c_limit & criteria2)]['mOIII']
err2  = tbl[np.where(c_limit & criteria2)]['dmOIII']

print(f'{galaxy.name}: literature {galaxy.mu:.3f}')
print(f'completeness limit = {galaxy.completeness_limit}')

fitter = MaximumLikelihood1D(pnlf,data1,mhigh=galaxy.completeness_limit)
mu1,dp1,dm1 = fitter([28])
fitter = MaximumLikelihood1D(pnlf,data2,mhigh=galaxy.completeness_limit)
mu2,dp2,dm2 = fitter([28])

print(f'plotting result for {galaxy.name} (binsize={binsize})')
filename = basedir / 'reports' / galaxy.name / f'{galaxy.name}_PNLF_with_SNR'
axes = plot_pnlf(tbl[criteria1]['mOIII'],mu1,galaxy.completeness_limit,binsize=binsize,mhigh=30,color=tab10[0])
axes = plot_pnlf(tbl[criteria2]['mOIII'],mu2,galaxy.completeness_limit,binsize=binsize,mhigh=30,filename=filename,color='grey',alpha=0.7,axes=axes)
plt.show()

In [ ]:
from pymuse.analyse import F

def N25(mu,completeness,data,deltaM):
    '''calculate the number of PN within deltaM
    
    the cutoff of the luminosity function is at mu-4.47.
    
    Step1: number of PN in data between cutoff and completeness
    Step2: calculate same number from theoretical function
    Step3: calculate theoreticla number betwenn cutoff and deltaM
    Step4: Scale number from Step1 with results from Step 2 and 3
    
    Parameters
    ----------
    mu : float
        distance modulus 
    completeness : float
        completeness limit (upper limit for PNLF). Used for normalization
    data : ndarray
        array of magnitudes
    deltaM : float
        Interval above the cutoff
    '''
    
    cutoff = mu - 4.47
    
    N_total  = len(data[data<completeness])
    p_deltaM = (F(cutoff+deltaM,mu) - F(cutoff,mu)) / (F(completeness,mu) - F(cutoff,mu))
    
    return N_total * p_deltaM

In [ ]:
import datetime
date = datetime.date.today().strftime('%Y.%m.%d')

row = [name,
       date,
       np.sum(criteria & c_limit),
       N25(mu1,galaxy.completeness_limit,data1,2.5),
       np.sum(c_shape & c_cut & (tbl['type']=='SNR') & c_limit),
       np.sum(c_shape & c_cut & c_SNR & c_limit),
       np.sum((c_shape & ~c_cut & (tbl['type']=='PN') & c_limit) | tbl['exclude']),
       cut,
       Distance(distmod=mu1).to(u.Mpc).value,
       mu1,dp1,dm1,
       mu2,dp2,dm2,
       np.sum(~mask)
       ]

results.loc[name] = row

# save results to output table
for col in results.colnames[2:]:
    if col.startswith('N_'):
        results[col].info.format = '%.0f'
    else:
        results[col].info.format = '%.3f'
        
with open(basedir/'data'/'interim'/ 'results.txt','w',newline='\n') as f:
    ascii.write(results,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

In [ ]:
# save results to output table
for col in results.colnames[2:]:
    if col.startswith('N_'):
        results[col].info.format = '%.0f'
    else:
        results[col].info.format = '%.3f'
        
results['err+d/Mpc'] = 2*np.log(10)*10**(results['(m-M)']/5) * results['err+(m-M)'] / 1e6
results['err-d/Mpc'] = 2*np.log(10)*10**(results['(m-M)']/5) * results['err-(m-M)'] / 1e6

with open(basedir/'data'/'interim'/ 'PHANGS_PNLF_distances.txt','w',newline='\n') as f:
    ascii.write(results[['name','date','(m-M)','err+(m-M)','err-(m-M)','d/Mpc','err+d/Mpc','err-d/Mpc']],f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

## Compare to stellar mass density

In [ ]:
sample_table = ascii.read(basedir/'data'/'catalogues'/'sample.txt')
sample_table.add_index('Name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

In [ ]:
x,y = sample_table.loc[name]['SkyCoord'].to_pixel(galaxy.wcs)
angle = 90-sample_table.loc[name]['posang']
eccentricity = np.sin(sample_table.loc[name]['Inclination']*u.deg).value
r25 = sample_table.loc[name]['r25']

In [ ]:
from photutils import EllipticalAnnulus
from regions import PixCoord,EllipseAnnulusPixelRegion

In [ ]:
def elliptical_aperture(center,eccentricity=1,angle=0*u.deg,a=1):
    
    if not 0<eccentricity<1:
        raise ValueError('only 0<eccentricity<1 permitted')
    
    fig = plt.figure(figsize=(two_column,two_column))
    ax  = fig.add_subplot(111,projection=galaxy.wcs)
    
    norm = simple_norm(galaxy.whitelight,'linear',clip=False,max_percent=95)
    ax.imshow(galaxy.whitelight,norm=norm,cmap=plt.cm.Greens)


    aperture = EllipseAnnulusPixelRegion(center,
                                 inner_width=a,
                                 inner_height=np.sqrt((a)**2 * (1-eccentricity**2)),
                                 outer_width=a*5,
                                 outer_height=np.sqrt((a*5)**2 * (1-eccentricity**2)),
                                 angle=angle)

    patch = aperture.as_artist(facecolor='none', edgecolor='black', lw=1,ls='--')
    ax.add_patch(patch)

    plt.show()
    
    return None

elliptical_aperture(PixCoord(x,y),
                     eccentricity=eccentricity,
                     angle=angle*u.deg,
                     a=r25)

In [ ]:
# 0.2 arcsec to rad
area_per_pixel = (9.6962736e-7*(Distance(distmod=parameters[name]['mu'])).to(u.kpc))**2

In [ ]:
seeing=Path('g:\Archive\MUSE\DR2\AUXILIARY\seeing_maps')

for file in [x for x in seeing.iterdir() if x.is_file()]:
    os.rename(file,(file.name.replace('_DR2','')

In [ ]:
np.nansum(galaxy.stellar_mass)*u.Msun/u.kpc**2*area_per_pixel

In [ ]:
data = tbl[np.where(criteria & c_limit)]

def sort_elliptical_bins(center,eccentricity=1,angle=0*u.deg,positions=None,n_bins=10,r_bins=100):
    
    if not 0<eccentricity<1:
        raise ValueError('only 0<eccentricity<1 permitted')
    
    fig = plt.figure(figsize=(two_column,two_column))
    ax  = fig.add_subplot(111,projection=galaxy.wcs)
    
    norm = simple_norm(galaxy.whitelight,'linear',clip=False,max_percent=95)
    ax.imshow(galaxy.whitelight,norm=norm,cmap=plt.cm.Greens)

    elliptical_bins = []
    bins = np.zeros(len(positions))-1
    radius = np.arange(r/2,r_bins*n_bins,r_bins)
    for n_bin in range(n_bins):
        
        aperture = EllipseAnnulusPixelRegion(center,
                                     inner_width=n_bin*r_bins,
                                     inner_height=np.sqrt((n_bin*r_bins)**2 * (1-eccentricity**2)),
                                     outer_width=(1+n_bin)*r_bins,
                                     outer_height=np.sqrt(((1+n_bin)*r_bins)**2 * (1-eccentricity**2)),
                                     angle=angle)
        #aperture.plot(ax=ax)
        patch = aperture.as_artist(facecolor='none', edgecolor='black', lw=1,ls='--')
        ax.add_patch(patch)
        
        bins[aperture.contains(PixCoord(data['x'],data['y']))] = n_bin
        
        elliptical_bins.append(aperture)
        
    plt.show()
    
    return elliptical_bins, bins, radius

elliptical_bins, bins, radius = sort_elliptical_bins(PixCoord(x,y),
                                                     eccentricity=eccentricity,
                                                     angle=angle*u.deg,
                                                     positions=PixCoord(data['x'],data['y']),
                                                     r_bins=200,
                                                     n_bins=7)

In [ ]:
density = []
for i,aperture in enumerate(elliptical_bins):
    try:
        area = np.sum(aperture.to_mask().multiply(~np.isnan(galaxy.stellar_mass)))
    except:
        area = aperture.area
    
    density.append(np.sum(bins==i)/area)
    
plt.scatter(radius,density)
plt.yscale('log')
plt.ylim([1e-5,1e-3])
plt.show()

In [ ]:
i=1
ring = elliptical_bins[i].to_mask().multiply(~np.isnan(galaxy.stellar_mass))
print(f'{elliptical_bins[i].area:.0f}')
print(f'{np.sum(ring)}')
plt.imshow(ring)
plt.show()

In [ ]:
sample_table

## Compare to existing Studies

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)
from astropy.table import vstack

from pymuse.load_references import NGC628, \
                                   pn_NGC628_kreckel, \
                                   snr_NGC628_kreckel, \
                                   pn_NGC628_herrmann, \
                                   NGC628_kreckel, \
                                   pn_NGC5068_herrmann, \
                                   pn_NGC3351_ciardullo, \
                                   pn_NGC3627_ciardullo


def get_fwhm(x,y):
    try:
        return galaxy.PSF[int(y),int(x)]
    except:
        return 0

for table in [NGC628,pn_NGC628_kreckel,snr_NGC628_kreckel,NGC628_kreckel,
              pn_NGC628_herrmann,pn_NGC5068_herrmann,pn_NGC3351_ciardullo,pn_NGC3627_ciardullo]:
    table['x'],table['y']= table['SkyCoord'].to_pixel(wcs=galaxy.wcs)
    table['fwhm'] = np.array([get_fwhm(x,y) for x,y in zip(table['x'],table['y'])])


exclude objects that lie outside our field of view

In [ ]:
# select the correct catalogue here
matchcoord = NGC628

catalogcoord = tbl[tbl['mOIII']<galaxy.completeness_limit].copy()
#catalogcoord['SkyCoord'] = 

matchcoord['in_frame'] = False
y_dim,x_dim = galaxy.shape

for row in matchcoord:
    x,y = row['x'], row['y']    
    if 0<=int(x)<x_dim and 0<=int(y)<y_dim:
        if not np.isnan(galaxy.PSF[int(y),int(x)]):
            row['in_frame'] = True
           
print(f"{np.sum(~matchcoord['in_frame'])} objects outside of our field of view")
matchcoord   = matchcoord[matchcoord['in_frame'] & (matchcoord['mOIII']<27.5)]

plot the detections from the paper and our own detections

In [ ]:
fig = plt.figure(figsize=(single_column,single_column))
ax1 = fig.add_subplot(111,projection=galaxy.wcs)

norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax1.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

ax1.scatter(matchcoord['x'],matchcoord['y'],marker='o',s=6,lw=0.2,edgecolor='tab:red',facecolors='none')
ax1.scatter(catalogcoord['x'],catalogcoord['y'],marker='o',s=6,lw=0.2,edgecolor='tab:orange',facecolors='none')

for row in matchcoord:
    txt,x,y = row['ID'], row['x']+5, row['y']    
    
    ax1.annotate(txt, (x, y),fontsize=4,color='tab:red')

plt.savefig(basedir / 'reports' / galaxy.name /f'{galaxy.name}_PN_position_comparison.pdf',dpi=600)

see how many match within 1"

In [ ]:
tolerance = '0.8"'
ID, angle, Quantity  = match_coordinates_sky(matchcoord['SkyCoord'],SkyCoord.from_pixel(catalogcoord['x'],catalogcoord['y'],galaxy.wcs))
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle[angle.__lt__(Angle(tolerance))].mean().to_string(u.arcsec,decimal=True)}')

In [ ]:
matchcoord['type'] = catalogcoord[ID]['type']
matchcoord['mOIIIF'] = catalogcoord[ID]['mOIII']

plt.scatter(matchcoord['mOIII'],matchcoord['mOIIIF'])
plt.plot([25.5,26.5],[25.5,26.5])
plt.xlim([25.5,26.5])
plt.ylim([25.5,26.5])
plt.show()

### Compare [OIII] and H$\alpha$ fluxes

In [ ]:
def compare_OIII_fluxes():
    #mpl.use('pgf')
    #plt.style.use('TeX.mplstyle')
    
    mpl.rcParams['pgf.preamble'] = [r'\usepackage[hidelinks]{hyperref}', ]
    
    matchcoord['mOIII_measured'] = catalogcoord[ID]['mOIII']
    matchcoord['dmOIII_measured'] = catalogcoord[ID]['dmOIII']
    
    crit = angle.__lt__(Angle("1s"))

    fig,ax = plt.subplots(figsize=(single_column,single_column))

    color=tab10[0]
    for s in ['Kreckel PN','Kreckel SNR','Herrmann PN']:
        color = next(ax._get_lines.prop_cycler)['color']
        tmp = matchcoord[(matchcoord['source']==s) & crit]
        plt.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
                     yerr = tmp['dmOIII_measured'],
                     marker='o',ms=2,ls='none',mec=color,mfc=color,ecolor=color,label=s)

    #plt.errorbar(matchcoord[crit]['mOIII'],matchcoord[crit]['mOIII_measured'],
    #             yerr = matchcoord[crit]['dmOIII_measured'],
    #             marker='o',ms=4,ls='none',mec=color,mfc=color,ecolor=color,label=s)

        
    base_url = 'https://ui.adsabs.harvard.edu/abs/'
    link = f'\href{{{base_url + matchcoord.meta["bibcode"]}}}{{{matchcoord.meta["reference"]}}}'

    xmin = np.floor(2*np.min(matchcoord['mOIII']))/2
    xmax = np.ceil(2*np.max(matchcoord['mOIII']))/2
    ymin = np.floor(2*np.min(matchcoord['mOIII_measured']))/2
    ymax = np.ceil(2*np.max(matchcoord['mOIII_measured']))/2
    
    plt.plot([xmin,xmax],[xmin,xmax],color='black',lw=0.4)
    plt.plot([xmin,xmax],[xmin-0.5,xmax-0.5],color='gray',lw=0.5,ls='--')
    plt.plot([xmin,xmax],[xmin+0.5,xmax+0.5],color='gray',lw=0.5,ls='--')
    #ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$' + f' {link}')
    ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ existing studies')
    ax.set_ylabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ this work')
    plt.legend()
    
    #plt.savefig(basedir / 'reports' / f'flux_comparison_OIII.pdf',dpi=600)
    plt.show()
    
compare_OIII_fluxes()

In [ ]:
def compare_HA_fluxes():
    
    #mpl.use('pgf')
    #plt.style.use('TeX.mplstyle')
    
    mpl.rcParams['pgf.preamble'] = [r'\usepackage[hidelinks]{hyperref}', ]
    
    catalogcoord['R2'] = catalogcoord['OIII5006'] / (catalogcoord['HA6562']+catalogcoord['NII6583'])
    catalogcoord['dR2'] = catalogcoord['R2']  * np.sqrt(catalogcoord['OIII5006_err']/catalogcoord['OIII5006_err']**2 + 1/(catalogcoord['HA6562']+catalogcoord['NII6583'])**2 * (catalogcoord['HA6562_err']**2+catalogcoord['NII6583_err']**2) )                                  
    
    matchcoord['R_measured'] = catalogcoord[ID]['R2']
    matchcoord['dR_measured'] = catalogcoord[ID]['dR2']

    crit = angle.__lt__(Angle("1s"))

    fig,ax = plt.subplots(figsize=(single_column,single_column))

    color=tab10[0]
    for s in ['Kreckel PN','Kreckel SNR','Herrmann PN']:
        color = next(ax._get_lines.prop_cycler)['color']
        tmp = matchcoord[(matchcoord['source']==s) & crit]
        print(f"{s}: {np.sum(tmp['R'] > tmp['R_measured']) / len(tmp) * 100:.2f} % under")
        plt.errorbar(tmp['R'],tmp['R_measured'],
                     #xerr = tmp['dR'],
                     #yerr = tmp['dR_measured'],
                     marker='o',ms=2,ls='none',mec=color,mfc=color,ecolor=color,label=s)

    xmin,xmax = 0,10
    ymin,ymax = 0,10
    
    plt.plot([xmin,xmax],[xmin,xmax],color='black',lw=0.4)
    plt.plot([xmin,xmax],[xmin-0.5,xmax-0.5],color='gray',lw=0.5,ls='--')
    plt.plot([xmin,xmax],[xmin+0.5,xmax+0.5],color='gray',lw=0.5,ls='--')
    ax.set_xlim([0,10])
    ax.set_ylim([0,10])
    #ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$' + f' {link}')
    ax.set_xlabel(r'[OIII]/(H$\alpha$+[NII]) existing studies')
    ax.set_ylabel(r'[OIII]/(H$\alpha$+[NII]) this study')
    plt.legend(loc=1)
    
    
    plt.savefig(basedir / 'reports' / f'flux_comparison_HA.pdf',dpi=600)
    
    plt.show()
    
catalogcoord = tbl[tbl['mOIII']<galaxy.completeness_limit]
compare_HA_fluxes()

### Francesco's Nebula catalogue

In [ ]:
# new catalogue
with fits.open(data_raw /'AUXILIARY'/'Nebulae catalogue' / 'Nebulae_Catalogue.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae=nebulae[(nebulae['gal_name']==galaxy.name) & (nebulae['flag_point_source']==2)]
nebulae['SkyCoord'] = SkyCoord.from_pixel(nebulae['cen_x'],nebulae['cen_y'],galaxy.wcs)
nebulae['mOIII'] = -2.5*np.log10(nebulae['OIII5006_FLUX']*1e-20) - 13.74
nebulae=nebulae[nebulae['mOIII']<galaxy.completeness_limit]

In [ ]:
with fits.open(basedir / 'data' / 'external' / 'nebula_catalogue_FS_v01.fits') as hdul:
    nebula_catalogue = Table(hdul[1].data)
    
PNe_candidate = nebula_catalogue[(nebula_catalogue['gal_name']==galaxy.name) & (nebula_catalogue['PNe_candidate']==1)]
PNe_candidate['SkyCoord'] = SkyCoord.from_pixel(PNe_candidate['cen_x'],PNe_candidate['cen_y'],galaxy.wcs)

In [ ]:
tolerance = '0.8"'
ID, angle, Quantity  = match_coordinates_sky(PNe_candidate['SkyCoord'],tbl['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'I recover {within_tolerance} of Francescos {len(angle)} sources')

In [ ]:
ID, angle, Quantity  = match_coordinates_sky(nebulae['SkyCoord'],tbl['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'Search in Francescos catalogue: {within_tolerance} of {len(angle)} of  match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')


In [ ]:
from photutils import CircularAperture

with fits.open(basedir / 'data' / 'external' / 'nebula_catalogue_FS_v01.fits') as hdul:
    nebula_catalogue = Table(hdul[1].data)

# (nebula_catalogue['region_size_pixels']<100) &
PNe_candidate = nebula_catalogue[(nebula_catalogue['gal_name']==galaxy.name) & (nebula_catalogue['PNe_candidate']==1)]
print(f'{len(PNe_candidate)} candidates from FS')

fig = plt.figure(figsize=(7,7))
ax  = fig.add_subplot(projection=galaxy.wcs)
norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

cat = tbl[c_PN & c_limit]

#positions = np.transpose([PNe_candidate['cen_x'],PNe_candidate['cen_y']])
#apertures = CircularAperture(positions, r=6)
ax.scatter(PNe_candidate['cen_x'],PNe_candidate['cen_y'],marker='o',s=5,lw=0.4,edgecolor='red',facecolors='none')
ax.scatter(cat['x'],cat['y'],marker='o',s=5,lw=0.4,edgecolor='blue',facecolors='none')

#apertures.plot(color='red',lw=.2, alpha=1)
plt.savefig(basedir / 'reports' / f'{galaxy.name}_FS_comparison.pdf',dpi=600)

In [ ]:
data = -2.5*np.log10(PNe_candidate['OIII5006_FLUX']*1e-20) - 13.74

fitter = MaximumLikelihood1D(pnlf,
                             data[data<28],
                             mhigh=galaxy.completeness_limit)
mu,dp,dm = fitter([24])
print(f'literature: {galaxy.mu:.2f}')
axes = plot_pnlf(data,
                 mu,
                 galaxy.completeness_limit,
                 binsize=0.4,
                 mhigh=30,
                 color=tab10[0])


### Enrico's Catalogue

In [ ]:
with fits.open(basedir / 'data' / 'external' / 'clumpfind_cat_v02.fits') as hdul:
    nebula_catalogue = Table(hdul[1].data)
nebula_catalogue = nebula_catalogue[(nebula_catalogue['gal_name']==galaxy.name)]

In [ ]:
from photutils import CircularAperture

with fits.open(basedir / 'data' / 'external' / 'clumpfind_cat_v02.fits') as hdul:
    nebula_catalogue = Table(hdul[1].data)
nebula_catalogue = nebula_catalogue[(nebula_catalogue['gal_name']==galaxy.name)]

nebula_catalogue.rename_column('cen_x','x')
nebula_catalogue.rename_column('cen_y','y')
nebula_catalogue = nebula_catalogue[~np.isnan(nebula_catalogue['x']) & ~np.isnan(nebula_catalogue['y'])]
nebula_catalogue['fwhm'] = np.array([galaxy.PSF[int(y),int(x)] for x,y in zip(nebula_catalogue['x'],nebula_catalogue['y'])])
nebula_catalogue['SkyCoord'] = SkyCoord.from_pixel(nebula_catalogue['x'],nebula_catalogue['y'],galaxy.wcs)
nebula_catalogue['mOIII'] = -2.5*np.log10(nebula_catalogue['OIII5006_FLUX']*1e-20) - 13.74


In [ ]:
ID, angle, Quantity  = match_coordinates_sky(nebula_catalogue['SkyCoord'],tbl['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} of  match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')


In [ ]:
nebula_catalogue['FHA'] = tbl[ID]['HA6562']

In [ ]:
match = nebula_catalogue[angle.__lt__(Angle(tolerance))]
plt.scatter(match['HA6562_FLUX'],match['FHA'])
plt.plot([0,1e6],[0,1e6])

#### Use flux measurements from Enrico

In [ ]:
for col in nebula_catalogue.colnames:
    if col.endswith('_FLUX'):
        nebula_catalogue.rename_column(col,col[:-5])
    if col.endswith('_FLUX_ERR'):
        nebula_catalogue.rename_column(col,col[:-9]+'_err')
        
nebula_catalogue['mOIII'] = -2.5*np.log10(nebula_catalogue['OIII5006']*1e-20) - 13.74
nebula_catalogue['dmOIII'] = np.abs( 2.5/np.log(10) * nebula_catalogue['OIII5006_err'] / nebula_catalogue['OIII5006'])

nebula_catalogue = emission_line_diagnostics(nebula_catalogue,galaxy.mu,galaxy.completeness_limit) 
pn_candidates = nebula_catalogue[nebula_catalogue['type']=='PN']

#### Measure flux with background subtraction

In [ ]:
flux = measure_flux(galaxy,
                    nebula_catalogue,
                    alpha=galaxy.alpha,
                    Rv=3.1,
                    Ebv=0.062,
                    extinction='MW',
                    aperture_size=1.5)

# calculate magnitudes from measured fluxes
flux['mOIII'] = -2.5*np.log10(flux['OIII5006']*1e-20) - 13.74
flux['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

emd = emission_line_diagnostics(flux,galaxy.mu,galaxy.completeness_limit) 
pn_candidates = emd[(emd['type']=='PN') & (emd['mOIII']<28)]


#### Visualize the result

In [ ]:
print(f'{len(pn_candidates)} nebulae from Enrico')

fig = plt.figure(figsize=(7,7))
ax  = fig.add_subplot(projection=galaxy.wcs)

norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

cat = tbl[c_PN & c_limit]

#positions = np.transpose([PNe_candidate['cen_x'],PNe_candidate['cen_y']])
#apertures = CircularAperture(positions, r=6)
ax.scatter(pn_candidates['x'],pn_candidates['y'],marker='o',s=4,lw=0.4,edgecolor='tab:orange',facecolors='none')
ax.scatter(cat['x'],cat['y'],marker='o',s=6,lw=0.4,edgecolor='tab:blue',facecolors='none')

#apertures.plot(color='red',lw=.2, alpha=1)
plt.savefig(basedir / 'reports' / f'{galaxy.name}_Enrico_comparison.pdf',dpi=600)

In [ ]:
from astropy.coordinates import match_coordinates_sky

matchcoord   = pn_candidates
matchcoord['SkyCoord'] = SkyCoord.from_pixel(matchcoord['x'],matchcoord['y'],galaxy.wcs)
cat['SkyCoord'] = SkyCoord.from_pixel(cat['x'],cat['y'],galaxy.wcs)

tolerance = '2s'
ID, angle, Quantity  = match_coordinates_sky(matchcoord['SkyCoord'],cat['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle[angle.__lt__(Angle(tolerance))].mean().to_string(u.arcsec,decimal=True)}"')

In [ ]:
fitter = MaximumLikelihood1D(pnlf,
                             pn_candidates[(pn_candidates['mOIII']<28) & (pn_candidates['mOIII']>10)]['mOIII'],
                             mhigh=galaxy.completeness_limit)
mu,dp,dm = fitter([24])
print(f'literature: {galaxy.mu:.2f}')
axes = plot_pnlf(pn_candidates['mOIII'],
                 mu,
                 galaxy.completeness_limit,
                 binsize=0.4,
                 mhigh=30,
                 color=tab10[0])


## Distance in parsec

the measured distances are in the form of the distance modulus $\mu = m-M$ which is the difference between apparent and absolute magnitude. By defintion of the absolte magnitude, we can convert this number into a distance in pc
$$
d = 10^{\frac{\mu}{5}+1} = 10 \cdot \exp\left( \ln 10 \frac{\mu}{5} \right) \\
\delta d = \frac{\ln 10}{5} 10 \exp\left( \ln 10 \frac{\mu}{5} \right) \delta \mu = 0.2 \ln 10 \; d \; \delta \mu
$$

In [ ]:
def distance_modulus_to_parsec(mu,mu_err=np.array([])):
    
    d = 10 * np.exp(np.log(10)*mu/5)
    if len(mu_err) > 0:
        d_err = 0.2 * np.log(10) * d * mu_err
    print(f'd = ({d/1e6:.2f} + {d_err[0]/1e6:.2f} - {d_err[1]/1e6:.2f}) Mpc')
    
    return d, d_err

d,d_err = distance_modulus_to_parsec(30.033,np.array([0.014,0.015]))

In [ ]:
def area(mu,Npixel,inclination,**kwargs):
    '''Calculate the survey area from parameters
    
    one can also pass the parameter dict as
    area(**parameters[name])
    additional parameters will be ignored
    '''
    
    size_of_pixel = 0.2*u.arcsec

    distance = Distance(distmod=mu)
    pixel_area = (size_of_pixel/u.arcsec * distance/206265)**2
        
    return pixel_area.to(u.kpc**2) *Npixel / np.cos(inclination*u.deg)
    

area(**parameters['NGC0628'])

## Luminosity-specific planetary nebula number

In [ ]:
def measure_luminosity(img,distance):
    
    # calculate total flux in erg / s /cm2
    flux = np.nansum(img) * 1e-20 * u.erg / u.s / u.cm**2
    
    luminosity = flux * 4*np.pi * distance**2
    
    return luminosity.to(u.Lsun)
    
    
# add [mask] to remove excluded regions
measure_luminosity(galaxy.whitelight,Distance(distmod=galaxy.mu))
    

In [ ]:
alpha = N25(mu,completeness,data,2.5) / measure_luminosity(galaxy.whitelight,Distance(distmod=galaxy.mu))
alpha*1e8

## Compare with SDSS

In [ ]:
from astroquery.skyview import SkyView
from reproject import reproject_interp
from skimage.measure import find_contours

In [ ]:
try:
    img = SkyView.get_images(galaxy.name,survey=['SDSSi','SDSSr','SDSSg'],height=10*u.arcmin,width=10*u.arcmin)
    print('using SDSS data')
except:
    img = SkyView.get_images(galaxy.name,survey=['Mellinger Red','Mellinger Green','Mellinger Blue'],height=10*u.arcmin,width=10*u.arcmin)
    print('using Mellinger')
    
r = img[0][0].data
g = img[1][0].data
b = img[2][0].data

rgb=create_RGB(r,g,b,percentile=[98,98,98],weights=[0.8,0.8,0.8])

muse_sdss, footprint = reproject_interp((galaxy.whitelight,galaxy.wcs),img[0][0].header)
contours = find_contours(footprint,level=0.5)

In [ ]:
fig=plt.figure(figsize=(single_column,single_column))
ax=fig.add_subplot(projection=WCS(img[0][0].header))
for cont in contours:
    ax.plot(cont[:,1],cont[:,0],color='red',lw=0.5)
ax.imshow(rgb)
ax.set(xlabel='R.A. (J2000)',ylabel='Dec. (J2000)')
plt.savefig(basedir/'reports'/galaxy.name/f'{galaxy.name}_SDSS.pdf',dpi=1000)
plt.show()